In [1]:
# data import and cleaning
import numpy as np

with open(f"../data/data.txt", 'r', encoding='utf-8-sig') as f:
    file_list = f.readlines()

    pointer = 0
    for i, line in enumerate(file_list):
        file_list[i] = [float(item) for item in line.replace(' ', '').split(',')]

    # first line is number of warehouses
    num_warehouses = int(file_list[pointer][0])
    pointer += 1

    # second line is number of goods
    num_goods = int(file_list[pointer][0])
    pointer += 1

    # third line is number of retailers
    num_retailers = int(file_list[pointer][0])
    pointer += 1

    # fourth line is the capacity of each warehouse
    warehouse_capacities = [int(value) for value in file_list[pointer]]
    assert len(warehouse_capacities) == num_warehouses
    pointer += 1

    # next block is the demand of each retailer for each good and the shortage unit cost
    retailer_good_demand = np.ndarray(shape=(num_retailers, num_goods))
    retailer_good_shortage_cost = np.ndarray(shape=(num_retailers, num_goods))
    for line in file_list[pointer:pointer + num_retailers * num_goods]:
        retailer = int(line[0]) - 1
        good = int(line[1]) - 1
        demand = int(line[2])
        shortage_cost = line[3]
        retailer_good_demand[retailer, good] = demand
        retailer_good_shortage_cost[retailer, good] = shortage_cost

    pointer += num_retailers * num_goods

    # next line is the sizes of each of the goods
    good_sizes = file_list[pointer]
    assert len(good_sizes) == num_goods
    pointer += 1

    # next line is the holding cost of each good
    good_holding_costs = file_list[pointer]
    assert len(good_holding_costs) == num_goods
    pointer += 1

    # next block is the initial stock of each good at each warehouse
    # we initialise as zeroes in case the file doesn't include zero stock goods
    warehouse_good_initial_stock = np.zeros(shape=(num_warehouses, num_goods))
    for line in file_list[pointer:pointer + num_warehouses * num_goods]:
        warehouse = int(line[0]) - 1
        good = int(line[1]) - 1
        initial_stock = int(line[2])
        warehouse_good_initial_stock[warehouse, good] = initial_stock

    pointer += num_warehouses * num_goods

    # the last block is the unit transportation cost for each good between each retailer and each warehouse
    # we initialise as zeroes in case the file doesn't include goods with zero transportation costs
    transportation_costs = np.zeros(shape=(num_retailers, num_warehouses, num_goods))
    for line in file_list[pointer:pointer + num_warehouses * num_retailers * num_goods]:
        warehouse = int(line[0]) - 1
        retailer = int(line[1]) - 1
        good = int(line[2]) - 1
        transport_cost = line[3]
        transportation_costs[warehouse, retailer, good] = transport_cost

print(f"""Data loaded successfully!
    - num_warehouses: {num_warehouses}
    - num_goods: {num_goods}
    - num_retailers {num_retailers}
    - warehouse_capacities: {warehouse_capacities}
    - retailer_good_demand:\n {retailer_good_demand}
    - retailer_good_shortage_cost:\n {retailer_good_shortage_cost}
    - good_sizes: {good_sizes}
    - good_holding_costs: {good_holding_costs}
    - warehouse_good_initial_stock:\n {warehouse_good_initial_stock}
    - transportation_costs:\n {transportation_costs}
""")

Data loaded successfully!
    - num_warehouses: 5
    - num_goods: 3
    - num_retailers 5
    - warehouse_capacities: [800, 1000, 700, 1000, 600]
    - retailer_good_demand:
 [[200. 150. 350.]
 [250. 200. 400.]
 [150. 300. 200.]
 [300. 150. 150.]
 [200. 100. 200.]]
    - retailer_good_shortage_cost:
 [[2.5 3.  2. ]
 [3.  2.  3. ]
 [2.  1.5 1.5]
 [1.5 3.  3. ]
 [2.5 2.5 2.5]]
    - good_sizes: [1.0, 0.8, 0.6]
    - good_holding_costs: [2.0, 5.0, 3.0]
    - warehouse_good_initial_stock:
 [[300. 200. 300.]
 [400. 300. 300.]
 [200. 100. 200.]
 [ 50. 150. 300.]
 [150. 150. 200.]]
    - transportation_costs:
 [[[0.6 0.8 1. ]
  [0.7 0.9 1. ]
  [1.3 1.5 0.8]
  [1.4 1.3 2. ]
  [1.8 1.6 1.2]]

 [[2.2 1.3 1.6]
  [1.9 2.1 2.5]
  [1.  2.5 1.5]
  [1.8 1.7 1.4]
  [1.2 1.9 2.3]]

 [[2.5 0.7 0.8]
  [0.4 1.3 1.4]
  [0.9 1.7 2.3]
  [1.8 2.3 1.8]
  [0.9 1.4 2.4]]

 [[1.6 1.9 0.6]
  [1.4 2.1 2. ]
  [0.6 1.  1.7]
  [1.5 2.2 2.6]
  [1.  1.6 1.9]]

 [[1.8 0.7 0.5]
  [1.8 1.4 2.1]
  [2.5 1.8 1.7]
  [2.6 1.8 2

In [10]:
import gurobipy as gp

m = gp.Model()
num_periods = 3

warehouses = range(num_warehouses)
goods = range(num_goods)
retailers = range(num_retailers)
periods = range(num_periods)

new_warehouse_stock = m.addMVar((num_warehouses, num_goods, num_periods), lb=0, ub=10000, vtype='I')
carried_warehouse_stock = m.addMVar((num_warehouses, num_goods, num_periods+1), lb=0, ub=10000, vtype='I')
transport = m.addMVar((num_warehouses, num_retailers, num_goods, num_periods), lb=0, ub=10000, vtype='I')
carried_retailer_stock = m.addMVar((num_retailers, num_goods, num_periods+1), lb=0, ub=10000, vtype='I')
short_retailer_stock = m.addMVar((num_retailers, num_goods, num_periods), lb=0, ub=10000, vtype='I')


m.setObjective(
    gp.quicksum(transport[w, r, g, p] * transportation_costs[w, r, g] for w in warehouses for r in retailers for g in goods for p in periods) +
    gp.quicksum(short_retailer_stock[r, g, p] * retailer_good_shortage_cost[r, g] for g in goods for r in retailers for p in periods) +
    gp.quicksum(carried_retailer_stock[r, g, p] * good_holding_costs[g] for g in goods for r in retailers for p in periods) +
    gp.quicksum(carried_warehouse_stock[w, g, p] * good_holding_costs[g] for g in goods for w in warehouses for p in periods),
    gp.GRB.MINIMIZE
)

# total amount goods at each warehouse cannot exceed capacity
m.addConstrs((gp.quicksum((new_warehouse_stock[w, g, p] + carried_warehouse_stock[w, g, p]) * good_sizes[g] for g in goods) <= warehouse_capacities[w] for w in warehouses for p in periods), name="WarehouseCapacities")

# first period carried warehouse stock equal to existing stock
m.addConstrs((carried_warehouse_stock[w, g, 1] == warehouse_good_initial_stock[w, g] for w in warehouses for g in goods), name="InitialWarehouseStock")

# first period carried retailer stock equal to 0
m.addConstrs((carried_retailer_stock[r, g, 1] == 0 for r in retailers for g in goods), name="InitialRetailerStock")

# amount of warehouse goods carried each period equal to the amount unsent
m.addConstrs(
    (
        carried_warehouse_stock[w, g, p + 1] >= new_warehouse_stock[w, g, p] + carried_warehouse_stock[w, g, p] - gp.quicksum(transport[w, r, g, p] for r in retailers) for w in warehouses for g in goods for p in periods
    ), name="WarehouseCarryAmounts"
)

# amount of retailer goods carried each period equal to the excess demand in each period
m.addConstrs(
    (
        carried_retailer_stock[r, g, p + 1] >= gp.quicksum(transport[w, r, g, p] for w in warehouses) - retailer_good_demand[r, g] + carried_retailer_stock[r, g, p] for r in retailers for g in goods for p in periods
    ), name="RetailerCarryAmounts"
)

# amount of retailer goods short of demand is equal to the unmet demand in each period
m.addConstrs(
    (
        short_retailer_stock[r, g, p] >= retailer_good_demand[r, g] - carried_retailer_stock[r, g, p] - gp.quicksum(transport[w, r, g, p] for w in warehouses) for r in retailers for g in goods for p in periods
    ), name="RetailerShortAmounts"
)

# total amount of good sent from warehouse must not exceed stock
m.addConstrs((gp.quicksum(transport[w, r, g, p] for r in retailers) <= new_warehouse_stock[w, g, p] + carried_warehouse_stock[w, g, p] for w in warehouses for g in goods for p in periods), name="TransportCannotExceedStock")

m.optimize()
# modelling stuff goes here

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 9 7950X 16-Core Processor, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 225 rows, 435 columns and 1425 nonzeros
Model fingerprint: 0x6cd05879
Variable types: 0 continuous, 435 integer (0 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+00]
  Objective range  [4e-01, 5e+00]
  Bounds range     [1e+04, 1e+04]
  RHS range        [5e+01, 1e+03]
Found heuristic solution: objective 37375.000000
Presolve removed 125 rows and 218 columns
Presolve time: 0.00s
Presolved: 100 rows, 217 columns, 590 nonzeros
Found heuristic solution: objective 32180.000000
Variable types: 0 continuous, 217 integer (0 binary)

Root relaxation: objective 2.063500e+04, 65 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  D

In [12]:
print(f"""
Results:
    - new_warehouse_stock: {[item.x for item in new_warehouse_stock]}
    - carried_warehouse_stock: {[item.x for item in carried_warehouse_stock]}
    - carried_retailer_stock: {[item.x for item in carried_retailer_stock]}
    - short_retailer_stock: {[item.x for item in short_retailer_stock]}
    - transport: {[item.x for item in transport]}
""")


Results:
    - new_warehouse_stock: [array([[200.,  -0., 200.],
       [150.,  -0., 150.],
       [800.,  -0., 800.]]), array([[  0.,  -0.,  -0.],
       [150.,  -0., 150.],
       [150.,  -0., 150.]]), array([[450.,  -0., 450.],
       [300.,  -0., 150.],
       [ -0.,  -0.,  -0.]]), array([[150.,  -0., 150.],
       [300.,   0., 300.],
       [  0.,  -0.,  -0.]]), array([[  0.,  -0.,  -0.],
       [  0.,  -0., 150.],
       [350.,  -0., 350.]])]
    - carried_warehouse_stock: [array([[   -0.,   300.,    -0., 10000.],
       [   -0.,   200.,    -0., 10000.],
       [   -0.,   300.,    -0., 10000.]]), array([[   -0.,   400.,    -0., 10000.],
       [   -0.,   300.,    -0., 10000.],
       [   -0.,   300.,    -0., 10000.]]), array([[   -0.,   200.,    -0., 10000.],
       [   -0.,   100.,    -0., 10000.],
       [   -0.,   200.,    -0., 10000.]]), array([[   -0.,    50.,    -0., 10000.],
       [   -0.,   150.,    -0., 10000.],
       [   -0.,   300.,    -0., 10000.]]), array([[   -0.,